In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

In [7]:
#로컬 모듈 import
import sys
import os
print(os.getcwd())
# 모듈 경로를 sys.path에 추가
module_path = os.path.abspath(os.path.join('..', 'my_transformer'))
if module_path not in sys.path:
    sys.path.append(module_path)
from my_transformer.my_transformer import Transformer

/root/25-1_DS_assignment/Week_01/25-1-DS-Week-1-Assignment


In [8]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import wandb

# 데이터셋 정의
class PatternDataset(Dataset):
    def __init__(self, num_samples=1000, sequence_length=4, max_num=10):
        self.data = []
        self.targets = []
        for _ in range(num_samples):
            start = torch.randint(0, max_num, (1,)).item()  # 시작 숫자
            diff = torch.randint(1, 5, (1,)).item()  # 등차
            sequence = [start + i * diff for i in range(sequence_length)]
            next_value = sequence[-1] + diff  # 다음에 올 숫자
            
            # 입력 시퀀스는 정수로 이루어진 시퀀스, 타겟은 다음에 올 숫자
            self.data.append(torch.tensor(sequence))
            self.targets.append(next_value)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

# 하이퍼파라미터 설정
src_vocab_size = 100
tgt_vocab_size = 200
d_model = 32
n_heads = 8
d_ff = 64
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.1
batch_size = 16
num_epochs = 50
learning_rate = 0.001

# WandB 초기화
#TODO
# https://wandb.ai/authorize (API Key 확인하는 웹사이트)
wandb.init(project="transformer", config={
    "src_vocab_size": src_vocab_size,
    "tgt_vocab_size": tgt_vocab_size,
    "d_model": d_model,
    "n_heads": n_heads,
    "d_ff": d_ff,
    "num_encoder_layers": num_encoder_layers,
    "num_decoder_layers": num_decoder_layers,
    "dropout": dropout,
    "batch_size": batch_size,
    "num_epochs": num_epochs,
    "learning_rate": learning_rate,
})

# 데이터셋 및 데이터로더 생성
dataset = PatternDataset(num_samples=1000)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 모델 초기화 (Transformer 모델이 별도로 정의되어 있어야 합니다)
model = Transformer(src_vocab_size, tgt_vocab_size, d_model, n_heads, d_ff, num_encoder_layers, num_decoder_layers, dropout)

# 손실 함수 및 최적화 도구 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 루프

def train():
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for src, tgt in dataloader:
            # 예측 시작을 위한 빈 타겟 시퀀스 준비
            tgt_input = torch.zeros_like(tgt).unsqueeze(1)
            tgt = tgt.unsqueeze(1)
            
            optimizer.zero_grad()
            output = model(src, tgt_input)
            
            # 출력 크기 조정 및 손실 계산
            output = output.view(-1, tgt_vocab_size)
            tgt = tgt.view(-1)
            
            loss = criterion(output, tgt)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
        
        # WandB에 손실값 로깅
        wandb.log({"epoch": epoch+1, "loss": avg_loss})

train()


Epoch 1/50, Loss: 2.2054
Epoch 2/50, Loss: 0.2660
Epoch 3/50, Loss: 0.0624
Epoch 4/50, Loss: 0.0249
Epoch 5/50, Loss: 0.0165
Epoch 6/50, Loss: 0.0323
Epoch 7/50, Loss: 0.0365
Epoch 8/50, Loss: 0.0084
Epoch 9/50, Loss: 0.0065
Epoch 10/50, Loss: 0.0043
Epoch 11/50, Loss: 0.0234
Epoch 12/50, Loss: 0.0789
Epoch 13/50, Loss: 0.0191
Epoch 14/50, Loss: 0.0093
Epoch 15/50, Loss: 0.0130
Epoch 16/50, Loss: 0.0242
Epoch 17/50, Loss: 0.0024
Epoch 18/50, Loss: 0.0029
Epoch 19/50, Loss: 0.0013
Epoch 20/50, Loss: 0.0007
Epoch 21/50, Loss: 0.0006
Epoch 22/50, Loss: 0.0005
Epoch 23/50, Loss: 0.0008
Epoch 24/50, Loss: 0.0004
Epoch 25/50, Loss: 0.0004
Epoch 26/50, Loss: 0.0005
Epoch 27/50, Loss: 0.0003
Epoch 28/50, Loss: 0.0003
Epoch 29/50, Loss: 0.0002
Epoch 30/50, Loss: 0.0002
Epoch 31/50, Loss: 0.0002
Epoch 32/50, Loss: 0.0002
Epoch 33/50, Loss: 0.0002
Epoch 34/50, Loss: 0.0004
Epoch 35/50, Loss: 0.0004
Epoch 36/50, Loss: 0.0002
Epoch 37/50, Loss: 0.0003
Epoch 38/50, Loss: 0.0002
Epoch 39/50, Loss: 0.

In [10]:
# 테스트 데이터셋 생성 (학습 데이터와 동일한 방식으로 생성)
test_dataset = PatternDataset(num_samples=1000)  # 테스트용 샘플 수
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

def test_model(model, dataloader):
    model.eval()  # 평가 모드로 설정
    total_correct = 0
    total_samples = 0

    with torch.no_grad():  # 그래디언트 계산 중지 (평가 시에는 필요하지 않음)
        for src, tgt in dataloader:
            # 입력 시퀀스 준비
            tgt_input = torch.zeros_like(tgt).unsqueeze(1)  # 예측 시작을 위한 빈 타겟 시퀀스
            tgt = tgt.unsqueeze(1)  # 타겟을 2D 텐서로 변환
            
            # 모델에 입력 시퀀스를 전달하고 예측 값 생성
            output = model(src, tgt_input)
            predicted = output.argmax(dim=-1)  # 예측 결과는 argmax를 통해 얻음
            
            # 실제 타겟과 예측값 비교
            correct = (predicted.view(-1) == tgt.view(-1)).sum().item()
            total_correct += correct
            total_samples += tgt.size(0)

    # 정확도 계산
    accuracy = total_correct / total_samples * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    
    # WandB에 테스트 정확도 로깅
    wandb.log({"test_accuracy": accuracy})
    wandb.run.summary["test_accuracy"] = accuracy

# 학습된 모델 테스트
test_model(model, test_dataloader)


Test Accuracy: 100.00%


Wandb Sweep 사용해보기!

In [6]:
sweep_config = {
    "method": "grid",  # "random", "bayes", "grid" 등 사용 가능
    "metric": {
        "name": "loss",
        "goal": "minimize"
    },
    "parameters": {
        "learning_rate": {
            "values": [0.0001, 0.0005, 0.001]
        },
        "batch_size": {
            "values": [16, 32, 64]
        },
        "d_model": {
            "values": [32, 64]
        }
    }
}

# Sweep 등록 (프로젝트 이름은 wandb.init에서 사용한 것과 동일해야 함)
sweep_id = wandb.sweep(sweep_config, project="transformer")
# 지정한 횟수만큼 에이전트를 실행 (count를 조절하여 실험 횟수를 늘릴 수 있습니다)
wandb.agent(sweep_id, function=train, count=10)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: ls7dov4q
Sweep URL: https://wandb.ai/kkyu0215-yonsei-university/transformer/sweeps/ls7dov4q


wandb: Agent Starting Run: xlvrmp2e with config:
wandb: 	batch_size: 16
wandb: 	d_model: 32
wandb: 	learning_rate: 0.0001


wandb: 
wandb: 🚀 View run northern-waterfall-2 at: https://wandb.ai/kkyu0215-yonsei-university/transformer/runs/b9eqq3l7
wandb: Find logs at: wandb/run-20250316_122609-b9eqq3l7/logs


Exception in thread Thread-11:
Traceback (most recent call last):
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/tmp/ipykernel_5251/1180014963.py", line 98, in train
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 449, in wrapper
    return func(self, *args, **kwargs)
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 401, in wrapper_fn
    return func(self, *args, **kwargs)
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 391, in wrapper
    return func(self, *args, **kwargs)
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 1873, in log
    self._log(data=data, step=step, commit=commit)
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 1587, in _log
    self._partial_history_callback(data, 

...

wandb: Agent Starting Run: azoteri5 with config:
wandb: 	batch_size: 32
wandb: 	d_model: 32
wandb: 	learning_rate: 0.001
Exception in thread Thread-19:
Traceback (most recent call last):
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/tmp/ipykernel_5251/1180014963.py", line 98, in train
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 449, in wrapper
    return func(self, *args, **kwargs)
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 401, in wrapper_fn
    return func(self, *args, **kwargs)
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 391, in wrapper
    return func(self, *args, **kwargs)
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 1873, in log
    self._log(data=data, step=step, commit=commit)
  File "/root/miniconda3/

Epoch 1/50, Loss: 0.0074


wandb: Agent Starting Run: 9ka4mhuq with config:
wandb: 	batch_size: 32
wandb: 	d_model: 64
wandb: 	learning_rate: 0.0001
Exception in thread Thread-20:
Traceback (most recent call last):
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/tmp/ipykernel_5251/1180014963.py", line 98, in train
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 449, in wrapper
    return func(self, *args, **kwargs)
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 401, in wrapper_fn
    return func(self, *args, **kwargs)
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 391, in wrapper
    return func(self, *args, **kwargs)
  File "/root/miniconda3/envs/DS/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 1873, in log
    self._log(data=data, step=step, commit=commit)
  File "/root/miniconda3

Epoch 1/50, Loss: 0.0065
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f213d350b50>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f206934f940, execution_count=6 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f206934fc10, raw_cell="sweep_config = {
    "method": "grid",  # "random".." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Bkyu/root/25-1_DS_assignment/Week_01/25-1-DS-Week-1-Assignment/main.ipynb#W5sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe